In [1]:
import time
import nbimporter

In [2]:
import utils
import blockchain as blockchain
import elliptic_encryption as encryption

Importing Jupyter notebook from utils.ipynb
Importing Jupyter notebook from blockchain.ipynb
Importing Jupyter notebook from elliptic_encryption.ipynb


### LOAD RESOURCES

In [3]:
settings = utils.load_yaml('resources/settings.yaml')

In [4]:
latest = utils.load_json('resources/latest.json')

### CONNECT TO ETHEREUM GATEWAY

In [5]:
web3 = blockchain.connect(settings)

### SERIALIZE THE SMART CONTRACT

In [6]:
auth_manager = blockchain.contract(latest['authmanager'], web3, settings)

### GENERATE & ASSIGN RSA ENCRYPTION KEY

In [7]:
def generate_key(device):
    
    # PRINT MESSAGE
    print('GENERATING NEW KEY')
    
    # GENERATE AN RSA ENCRYPTION KEYPAIR
    private_key, public_key = encryption.generate_keys()
    
    # ASSIGN THE PUBLIC KEY TO THE DEVICE
    auth_manager.write({
        'func': 'assign_key',
        'params': [device, public_key]
    })
    
    # STORE THE PRIVATE KEY AS A FILE
    utils.save_data(private_key, 'data/keys/' + public_key + '.key')
    
    # PRINT MESSAGE
    print('FINISHED')

### FETCH MOST RECENT BACKLOG

In [8]:
def fetch_backlog():
    
    # FETCH RAW BACKLOG
    raw_backlog = auth_manager.read('fetch_backlog')
    
    # RETURN FILTERED BACKLOG
    return utils.filter_backlog(raw_backlog)

### MODIFICATION EVENT

In [9]:
modification_event = auth_manager.event('modification')

### EVENT LOOP

In [10]:
try:
    print('AWAITING REQUESTS...\n')
    
    # FETCH BACKLOG
    backlog = fetch_backlog()

    while(True):

        # REFRESH BACKLOG WHEN NOTIFIED
        for event in modification_event.get_new_entries():
            backlog = fetch_backlog()

        # PERFORM TASKS IN BACKLOG
        for device in backlog:
            generate_key(device)
            
        # SLEEP FOR A SECOND
        time.sleep(1)

except:
    print('\nTHE PROCESS WAS MANUALLY KILLED')

AWAITING REQUESTS...

GENERATING NEW KEY
FINISHED

THE PROCESS WAS MANUALLY KILLED
